## Example of how to find the nearest green area using the GreenAreaFinderCopernicus and GreenAreaFinderOSM class

#### Find the green area details for Copernicus

In [1]:
from data.boundingbox.BoundingBox import BoundingBox
from data.downloader.CopernicusDownloader import CopernicusDownloader
from data.downloader.OSMDownloader import OSMDownloader
from data.utils.LandUtils import LandUtils
from data.utils.GreenAreaFinderCopernicus import GreenAreaFinderCopernicus
from data.downloader.GHSPOPDownloader import GHSPOPDownloader
from data.utils.LandUtilsOSM import LandUtilsOSM
from data.utils.LandUtilsCopernicus import LandUtilsCopernicus
import json

copernicus_downloader = CopernicusDownloader(
        client_id="sh-a24a739d-d123-419b-a409-81a190c436c2",
        client_secret="dcUnE32uBB1gLvlxyi3qOeUPdGGpNMRs",
        token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
        use_oidc=False
    )
osm_downloader = OSMDownloader()
ghspop = GHSPOPDownloader(shapefile="../../tiling_schema/WGS84_tile_schema.shp")

bbox = BoundingBox()
bounding_box = bbox.get_bounding_box(query="Piazza Castello, Torino", method="from_center_radius", radius_km=15)
# we are using the "from_center_radius" method to get the bounding box coordinates, it gets the bounding box from a center point and a radius in km
# different methods are available to get the bounding box coordinates, such as "from_coordinates" and "from_geojson"

copernicus_area = copernicus_downloader.get_data(bounding_box)

osm_area = osm_downloader.get_data(bounding_box)

ghspop_area = ghspop.get_data(bounding_box)

osm_utils = LandUtilsOSM(osm_area, bounding_box)
osm_traffic = osm_utils.get_traffic_area("walk") # get the traffic area from the OSM data with the network_type "walk"

address = "Via Principessa Clotilde 28/B, Torino"
lat, lon = LandUtils().get_coordinates_from_address(address)
print(f"Coordinates for the starting point: {lat}, {lon}")

osm_green = osm_utils.get_green_area() # get the green area from the OSM data
osm_green_raster = LandUtils().vector_to_raster(osm_green, copernicus_area) # convert the vector green area to raster

copernicus_green = LandUtilsCopernicus(copernicus_area, osm_green_raster)
copernicus_green = copernicus_green.get_green_area() # get the green area from the Copernicus data

green_area_finder_copernicus = GreenAreaFinderCopernicus(
    copernicus_green,
    osm_traffic,
    ghspop_area
)

population_green_ratio = green_area_finder_copernicus.green_area_per_person()
print("-----------COPERNICUS-----------")
population_green_ratio = json.loads(population_green_ratio)
population_green_ratio = population_green_ratio["green_area_per_person"]
print(f"Green area / population: {population_green_ratio}")
networtk_type = "walk"
isochrone_area = green_area_finder_copernicus.get_isochrone_green(lat, lon, 12, networtk_type) # get the green area reachable in a certain time from the starting point
isochrone_area = json.loads(isochrone_area)
time = isochrone_area.get("isochrone_time_minutes")
green_area_percentage = isochrone_area["green_area_percentage"]
areasq = isochrone_area["green_area_sqm"]
print(f"Time max to reach the green area for the network_type {networtk_type}: {time} minutes")
print(f"Green area percentage: {green_area_percentage}%")
print(f"Green area in sqm walked: {areasq} sqm")


directions = green_area_finder_copernicus.direction_to_green(lat, lon,networtk_type)
directions = json.loads(directions)
minute = directions['estimated_time_minutes']
distance_km = directions['distance_km']
print(f"Estimated distance for the network type {networtk_type} to the nearest green area:")
print(f"Estimated time to the nearest green area: {minute} minutes")
print(f"Distance to the nearest green area: {distance_km} km")

Authenticated using refresh token.
Coordinates for the starting point: 45.0813291, 7.6675035
-----------COPERNICUS-----------
Green area / population: 162.6347
Time max to reach the green area for the network_type walk: 12 minutes
Green area percentage: 9.61%
Green area in sqm walked: 7100 sqm
Estimated distance for the network type walk to the nearest green area:
Estimated time to the nearest green area: 4.6 minutes
Distance to the nearest green area: 0.3396 km


#### Find the green area details for OSM

In [1]:
from data.boundingbox.BoundingBox import BoundingBox
from data.downloader.CopernicusDownloader import CopernicusDownloader
from data.downloader.OSMDownloader import OSMDownloader
from data.utils.LandUtils import LandUtils
from data.utils.GreenAreaFinderOSM import GreenAreaFinderOSM
from data.downloader.GHSPOPDownloader import GHSPOPDownloader
from data.utils.LandUtilsOSM import LandUtilsOSM
import json

copernicus_downloader = CopernicusDownloader(
        client_id="sh-a24a739d-d123-419b-a409-81a190c436c2",
        client_secret="dcUnE32uBB1gLvlxyi3qOeUPdGGpNMRs",
        token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
        use_oidc=False
    )
osm_downloader = OSMDownloader()
ghspop = GHSPOPDownloader(shapefile="../../tiling_schema/WGS84_tile_schema.shp")

bbox = BoundingBox()
bounding_box = bbox.get_bounding_box(query="Piazza Castello, Torino", method="from_center_radius", radius_km=15)
# we are using the "from_center_radius" method to get the bounding box coordinates, it gets the bounding box from a center point and a radius in km
# different methods are available to get the bounding box coordinates, such as "from_coordinates" and "from_geojson"

copernicus_area = copernicus_downloader.get_data(bounding_box)

osm_area = osm_downloader.get_data(bounding_box)

ghspop_area = ghspop.get_data(bounding_box)

osm_utils = LandUtilsOSM(osm_area, bounding_box)
osm_traffic = osm_utils.get_traffic_area("walk") # get the traffic area from the OSM data with the network_type "walk"

address = "Via Principessa Clotilde 28/B, Torino"
lat, lon = LandUtils().get_coordinates_from_address(address)
print(f"Coordinates for the starting point: {lat}, {lon}")

osm_green = osm_utils.get_green_area() # get the green area from the OSM data
osm_green_raster = LandUtils().vector_to_raster(osm_green, copernicus_area) # convert the vector green area to raster

print("-----------OSM-----------")
green_area_finder_osm = GreenAreaFinderOSM(
    osm_green_raster,
    osm_traffic,
    ghspop_area,
)
population_green_osm_ratio = green_area_finder_osm.green_area_per_person()
population_green_ratio = json.loads(population_green_osm_ratio)
population_green_osm_ratio = population_green_ratio["green_area_per_person"]
print(f"Green area / population: {population_green_osm_ratio}")

network_type = "walk"
isochrone_area = green_area_finder_osm.get_isochrone_green(lat, lon, 12, network_type)
isochrone_area = json.loads(isochrone_area)
time = isochrone_area.get("isochrone_time_minutes")
green_area_percentage = isochrone_area["green_area_percentage"]
areasq = isochrone_area["green_area_sqm"]
print(f"Time max to reach the green area for the network_type {network_type}: {time} minutes")
print(f"Green area percentage: {green_area_percentage}%")
print(f"Green area in sqm walked: {areasq} sqm")


directions = green_area_finder_osm.direction_to_green(lat, lon,network_type)
directions = json.loads(directions)
minute = directions['estimated_time_minutes']
distance_km = directions['distance_km']
print(f"Estimated distance for the network type {network_type} to the nearest green area:")
print(f"Estimated time to the nearest green area: {minute} minutes")
print(f"Distance to the nearest green area: {distance_km} km")

Authenticated using refresh token.
Coordinates for the starting point: 45.0813291, 7.6675035
-----------OSM-----------
Green area / population: 198.8779
Time max to reach the green area for the network_type walk: 12 minutes
Green area percentage: 11.23%
Green area in sqm walked: 8100 sqm
Estimated distance for the network type walk to the nearest green area:
Estimated time to the nearest green area: 0.9 minutes
Distance to the nearest green area: 0.0629 km
